In [ ]:
SMOKE_TEST = True
# SMOKE_TEST = False

# Number of models to train - None means all models (ignored in smoke test mode)
NUM_MODELS = None

SKIP_TRAINED = True

# Path to YAML config file containing list of model configs to train
CONFIG_LIST = 'notebooks/configs/mmocr_det_model_list.yml'

# Text Detection Training


In [2]:
import warnings

# Ignore all UserWarnings emitted from any submodule of torch
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module=r"torch.*"
)
# Ignore all UserWarnings emitted from any submodule of torch
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module=r"mmcv.*"
)

In [ ]:
import yaml
from pathlib import Path

# Load model configs from YAML file
with open(CONFIG_LIST, 'r') as f:
    config_paths = yaml.safe_load(f)

# Filter out commented lines and empty entries
active_configs = [cfg for cfg in config_paths if cfg and not cfg.strip().startswith('#')]

# Map model types to checkpoint URLs
CHECKPOINT_URLS = {
    "dbnetpp_custom": "https://download.openmmlab.com/mmocr/textdet/dbnetpp/dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015/dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015_20221101_124139-4ecb39ac.pth",
    "dbnet_custom": "https://download.openmmlab.com/mmocr/textdet/dbnet/dbnet_resnet50-oclip_1200e_icdar2015/dbnet_resnet50-oclip_1200e_icdar2015_20221102_115917-bde8c87a.pth",
    "psenet_custom": "https://download.openmmlab.com/mmocr/textdet/psenet/psenet_resnet50-oclip_fpnf_600e_icdar2015/psenet_resnet50-oclip_fpnf_600e_icdar2015_20221101_131357-2bdca389.pth",
    "panet_custom": "https://download.openmmlab.com/mmocr/textdet/panet/panet_resnet18_fpem-ffm_600e_icdar2015/panet_resnet18_fpem-ffm_600e_icdar2015_20220826_144817-be2acdb4.pth",
    "textsnake_custom": "https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500_20221101_134814-a216e5b2.pth",
    "fcenet_custom": "https://download.openmmlab.com/mmocr/textdet/fcenet/fcenet_resnet50-oclip_fpn_1500e_icdar2015/fcenet_resnet50-oclip_fpn_1500e_icdar2015_20221101_150145-5a6fc412.pth"
}

# Create mapping from config paths to checkpoint URLs
CONFIG_TO_CKPT = {}
for config_path in active_configs:
    config_path_obj = Path(config_path)
    # Extract model type from parent directory name
    model_type = config_path_obj.parent.name
    if model_type in CHECKPOINT_URLS:
        CONFIG_TO_CKPT[config_path] = CHECKPOINT_URLS[model_type]

print(f"Loaded {len(active_configs)} active configs:")
for config in active_configs:
    print(f"  - {config}")

ROOT_CONFIG_FOLDER = 'configs/textdet'

In [ ]:
#@title Train single model

from pathlib import Path
from mmengine.runner import Runner
import time
from mmengine import Config
from dotenv import load_dotenv

if SMOKE_TEST:
    load_dotenv() # NOTE: make sure to reload notebook when changing .env to use new env variables
    %cd ~/bonting-identification

    if not active_configs:
        raise ValueError("No active configs found in CONFIG_LIST")
    
    # Use the first active config for smoke test
    model_config = active_configs[0]
    
    cfg = Config.fromfile(model_config)
    cfg['load_from'] = CONFIG_TO_CKPT[model_config]
    cfg.visualizer.name = f'{time.localtime()}'

    # Optionally, smoke test on 1 epoch
    cfg.train_cfg['max_epochs'] = 1

    runner = Runner.from_cfg(cfg)
    result = runner.train()

In [ ]:
# !rm -rf work_dirs/*


In [ ]:
#@title Train all models

import os
from mmengine.runner import Runner
import time
from mmengine import Config
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

if not SMOKE_TEST:
    load_dotenv() # NOTE: make sure to reload notebook when changing .env to use new env variables
    %cd ~/bonting-identification

    results = []
    model_configs = []
    ckpts = []

    # Determine how many models to train
    if NUM_MODELS is None:
        # Use all models when NUM_MODELS is None
        models_to_train = len(active_configs)
    else:
        # Use specified number of models
        models_to_train = min(NUM_MODELS, len(active_configs))
    
    for model_config in active_configs[:models_to_train]:
        cfg = Config.fromfile(model_config)
        cfg['load_from'] = CONFIG_TO_CKPT[model_config]
        cfg.visualizer.name = f'{time.localtime()}'

        # cfg.train_cfg['max_epochs'] = 1

        runner = Runner.from_cfg(cfg)
        result = runner.train()

        results.append(result)
        model_configs.append(Path(model_config).name.rstrip('.py'))
        ckpts.append(Path(CONFIG_TO_CKPT[model_config]).parts[-2])


In [ ]:
# results_df = pd.DataFrame(results)
# results_df.insert(0, 'model_config', model_configs)
# results_df.insert(1, 'ckpt', ckpts)
# results_df = results_df.set_index(['model_config', 'ckpt'])
# results_df.sort_values('cegdr/hmean', ascending=False, inplace=True)
# results_df


In [ ]:
# save_path = Path('reports/eval/cegdr/textdet/mmocr_finetuned_det_results.csv')
# save_path.parent.mkdir(parents=True, exist_ok=True)
# print(f'Saving results to:\n{save_path}')
# results_df.to_csv(save_path, index=True, header=True)
